In [1]:
sc

In [5]:
import json 
import re
import os
import glob

In [6]:
def collection(df):
    subjectCollection = open('SubjectCollection.txt', 'a')
    personsCollection = open('PersonsCollection.txt', 'a')
    organizationCollection = open('OrganizationCollection.txt', 'a')
    #creativeCollection = open('CreativeCollection.txt', 'a')
    
    for i in range(len(df)):
        for j in range(len(df[i])):
            if df[i][j][1] == 'subject':
                subject = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                for word in subject.split():
                    subjectCollection.write(word+'\n')
                
            elif df[i][j][1] == 'persons':
                person = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                personsCollection.write(person+'\n')
            
            elif df[i][j][1] == 'organizations':
                person = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                organizationCollection.write(person+'\n')
# DONT NEED            
#             elif df[i][j][1] == 'creative_works':
#                 person = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
#                 creativeCollection.write(person+'\n')
            
#             else:
#                 print df[i][j][1]
                
    subjectCollection.close()
    personsCollection.close()
    organizationCollection.close()

In [7]:
#archive = open('archivejan1.json', 'r')
# os.listdir('~')
# df.show()
# df.printSchema()
cwd = os.getcwd()
print cwd
for i in  glob.glob(cwd+'/archive*'):
    df = spark.read.json(i)
    df = df.select('response.docs.keywords')
    df = df.first()[0]
    collection(df)

/Users/Enzo/anaconda2/CSC_599_WORK


In [41]:
# Will give error
print df[0]
print "======================================"
print df[0][0]
print len(df)
print len(df[0])

[Row(is_major=u'N', name=u'subject', rank=u'1', value=u'Cyberwarfare and Defense'), Row(is_major=u'N', name=u'subject', rank=u'2', value=u'Presidential Election of 2016'), Row(is_major=u'N', name=u'persons', rank=u'3', value=u'Merkel, Angela'), Row(is_major=u'N', name=u'persons', rank=u'4', value=u'Putin, Vladimir V')]
Row(is_major=u'N', name=u'subject', rank=u'1', value=u'Cyberwarfare and Defense')
5206
4


# WORKING WITH TEXT FILES

In [36]:
rddPersons = sc.textFile(cwd+"/PersonsCollection.txt")
rddOrganizations = sc.textFile(cwd+"/OrganizationCollection.txt")

print cwd

/Users/Enzo/anaconda2/CSC_599_WORK


### - SubjectCollection.txt

In [51]:
from nltk.corpus import stopwords

In [53]:
f = open(cwd+"/SubjectCollection.txt", 'r').read() # .read() turns text file into one big string

# word for word in f.split()
# filter words will be a list of words, incated by the first 'word'
filterWords = [word for word in f.split() if word not in stopwords.words('english')]
print filterWords

['mergers', 'acquisitions', 'divestitures', 'regulation', 'deregulation', 'industry', 'shareholder', 'rights', 'activism', 'united', 'states', 'economy', 'banking', 'financial', 'institutions', 'legal', 'profession', 'baseball', 'sports', 'injuries', 'psychology', 'psychologists', 'car', 'services', 'livery', 'cabs', 'labor', 'jobs', 'baseball', 'colleges', 'universities', 'home', 'appliances', 'vacuum', 'cleaners', 'robots', 'robotics', 'engineering', 'engineers', 'vacuum', 'cleaners', 'engineering', 'engineers', 'home', 'appliances', 'ferries', 'muslims', 'islam', 'terrorism', 'sunni', 'muslims', 'shiite', 'muslims', 'international', 'relations', 'appointments', 'executive', 'changes', 'museums', 'finances', 'appointments', 'executive', 'changes', 'boards', 'directors', 'art', 'united', 'states', 'politics', 'government', 'women', 'girls', 'united', 'states', 'international', 'relations', 'elections', 'house', 'representatives', 'women', 'rights', 'elections', 'senate', 'appointments

In [88]:
rddSubject = sc.parallelize(filterWords)

In [89]:
rddSubject.take(10)

['mergers',
 'acquisitions',
 'divestitures',
 'regulation',
 'deregulation',
 'industry',
 'shareholder',
 'rights',
 'activism',
 'united']

In [90]:
wcSubject = rddSubject.map(lambda x: (x,1)) \
         .reduceByKey(lambda x,y: x+y)

wcSubject.take(10)

[('mardi', 3),
 ('managed', 319),
 ('dance', 4),
 ('dollar', 7),
 ('alien', 1),
 ('swindling', 92),
 ('sleep', 21),
 ('lymphatic', 3),
 ('colon', 5),
 ('decisions', 89)]

In [91]:
import heapq

def mapper2(counts):
    yield heapq.nlargest(100, counts, key=lambda x: x[1])
    
def reducer2(top31,top32):
    return heapq.nlargest(100, top31 + top32, key=lambda x: x[1])


top100Subject = wcSubject.mapPartitions(mapper2).reduce(reducer2)

In [92]:
top100Subject[:10]

[('states', 3516),
 ('united', 3503),
 ('politics', 3197),
 ('government', 3183),
 ('fashion', 1005),
 ('international', 794),
 ('news', 762),
 ('immigration', 750),
 ('books', 742),
 ('literature', 720)]

In [98]:
TopSubjects = open("Top100Subjects.txt", 'a')

for sub in top100Subject:
    TopSubjects.write(sub[0])
    TopSubjects.write("\n")

TopSubjects.close()

### - PersonCollection.txt & Organizations.txt

In [73]:
print rddPersons.take(10)
print "=================="
print rddOrganizations.take(10)

[u'trump  donald j', u'strine  leo e jr', u'syndergaard  noah', u'montero  rafael', u'collins  terry l', u'soros  george', u'orban  viktor', u'de blasio  bill', u'campbell  thomas p', u'coburn  erin']
[u'tulane university', u'major league baseball', u'new york mets', u'uber technologies inc', u'new york mets', u'central european university', u'dyson', u'dyson', u'hornblower cruises and events', u'dawn  pakistani newspaper ']


In [74]:
wcPeople = rddPersons.map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y: x+y)

wcOrganizations = rddOrganizations.map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y: x+y)
    
print wcPeople.take(10)
print "==============="
print wcOrganizations.take(10)

[(u'reider  jonah', 1), (u'shannon  christopher', 1), (u'litiatco  armando', 2), (u'zeid ra ad zeid al hussein', 1), (u'magoon  daniel j', 1), (u'chozick  amy', 1), (u'wells  jane', 2), (u'afineevsky  evgeny         ', 2), (u'browder  ben', 1), (u'garrett  myles         ', 3)]
[(u'white stripes  music group ', 1), (u'edun', 1), (u'association of international photography art dealers', 1), (u'cafe china  manhattan  ny  restaurant ', 1), (u'office of the comptroller  nyc ', 2), (u'primary stages co', 2), (u'chloe  fashion label ', 7), (u'grocery manufacturers assn', 1), (u'tom cat bakery inc', 1), (u'mckinsey global institute', 2)]


In [75]:
import heapq

def mapper2(counts):
    yield heapq.nlargest(100, counts, key=lambda x: x[1])
    
def reducer2(top31,top32):
    return heapq.nlargest(100, top31 + top32, key=lambda x: x[1])


top100People = wcPeople.mapPartitions(mapper2).reduce(reducer2)

top100Organizations = wcOrganizations.mapPartitions(mapper2).reduce(reducer2)

In [76]:
print top100People[:10]
print "=========="
print top100Organizations[:10]

[(u'trump  donald j', 3252), (u'obama  barack', 292), (u'gorsuch  neil m', 133), (u'putin  vladimir v', 133), (u'sessions  jefferson b iii', 120), (u'de blasio  bill', 117), (u'tillerson  rex w', 89), (u'cuomo  andrew m', 88), (u'assad  bashar al ', 88), (u'ryan  paul d jr', 84)]
[(u'republican party', 437), (u'house of representatives', 240), (u'senate', 230), (u'new york times', 209), (u'supreme court  us ', 202), (u'islamic state in iraq and syria  isis ', 182), (u'democratic party', 172), (u'justice department', 140), (u'european union', 126), (u'new york knicks', 105)]


#### Look for   =>   Top100People.txt => manually handled top 100 people

In [97]:
TopOrganizations = open("Top100Organizations.txt", 'a')

for sub in top100Organizations:
    TopOrganizations.write(sub[0])
    TopOrganizations.write("\n")

TopOrganizations.close()